In [1]:
import shap
import optuna
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
import gc
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.linear_model import LogisticRegression, SGDClassifier
import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation, reset_parameter
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import GridSearchCV, StratifiedKFold
import numpy as np
from time import time
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
import joblib

In [2]:
# Load your model
model_lgbm = joblib.load("/kaggle/input/models-level-1/model_lgbm.pickle")
model_xgb = joblib.load("/kaggle/input/models-level-1/model_xgb_good.pickle")
model_rdf = joblib.load("/kaggle/input/models-level-1/model_rdf.pickle")

[20:58:11] WARNING: /workspace/src/gbm/gbtree.cc:360: 
  Loading from a raw memory buffer (like pickle in Python, RDS in R) on a CPU-only
  machine. Consider using `save_model/load_model` instead. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.  Changing `tree_method` to `hist`.
[20:58:11] WARNING: /workspace/src/gbm/gbtree.cc:385: Changing updater from `grow_gpu_hist` to `grow_quantile_histmaker`.
[20:58:11] WARNING: /workspace/src/context.cc:44: No visible GPU is found, setting device to CPU.


In [3]:
df_train = pd.read_parquet("/kaggle/input/data-tour/train.parquet")
df_train

,id,rn,pre_since_opened,pre_since_confirmed,pre_pterm,pre_fterm,pre_till_pclose,pre_till_fclose,pre_loans_credit_limit,pre_loans_next_pay_summ,...,enc_paym_22,enc_paym_23,enc_paym_24,enc_loans_account_holder_type,enc_loans_credit_status,enc_loans_credit_type,enc_loans_account_cur,pclose_flag,fclose_flag,flag
0,1678548,3,8,7,17,16,9,1,9,2,...,0,0,1,1,3,2,1,0,0,0
1,2834188,11,3,2,0,7,14,8,2,5,...,0,0,1,1,2,4,1,0,0,0
2,811902,11,9,6,11,13,14,8,2,5,...,3,3,4,1,2,4,1,0,0,0
3,836450,1,16,0,13,0,4,9,5,2,...,3,3,4,1,3,4,1,0,0,0
4,1769024,15,9,9,4,8,1,11,1,2,...,3,3,4,1,2,4,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17659828,2103608,4,1,9,4,8,1,11,10,2,...,3,3,4,1,2,3,1,1,1,0
17659829,2169551,7,15,4,12,10,6,13,15,2,...,0,0,1,1,3,2,1,0,0,0
17659830,1613436,1,8,9,16,5,9,9,10,3,...,3,3,4,1,3,4,1,0,0,0
17659831,1604231,6,17,13,12,1,2,4,2,2,...,0,0,1,1,3,0,1,0,0,0


In [4]:
X_train, X_val, y_train, y_val = train_test_split(df_train.drop(["flag"],axis = 1), df_train.flag, stratify=df_train.flag, test_size=0.05, random_state = 42)
#del df_train
#_= gc.collect()

In [5]:
# Prédictions sur le jeu d'entraînement
train_pred_xgb = model_xgb.predict_proba(X_train)[:, 1]

In [6]:
train_auc_xgb = roc_auc_score(y_train, train_pred_xgb)
print(f"Train AUC: {train_auc_xgb}\n")

Train AUC: 0.6992972575340508



In [7]:
train_pred_lgbm = model_lgbm.predict_proba(X_train)[:, 1]

In [8]:
train_auc_lgbm = roc_auc_score(y_train, train_pred_lgbm)
print(f"Train AUC: {train_auc_lgbm}\n")

Train AUC: 0.6973139573473583



In [9]:
train_pred_rdf = model_rdf.predict_proba(X_train)[:, 1]

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:   41.5s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:  2.9min
[Parallel(n_jobs=4)]: Done 357 tasks      | elapsed:  6.7min
[Parallel(n_jobs=4)]: Done 600 out of 600 | elapsed: 11.4min finished


In [10]:
train_auc_rdf = roc_auc_score(y_train, train_pred_rdf)
print(f"Train AUC: {train_auc_rdf}\n")

Train AUC: 0.6287516433066563



In [11]:
# Prédictions sur le jeu de test
test_pred_xgb = model_xgb.predict_proba(X_val)[:, 1]

In [12]:
test_pred_lgbm = model_lgbm.predict_proba(X_val)[:, 1]

In [13]:
test_pred_rdf = model_rdf.predict_proba(X_val)[:, 1]

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    1.9s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:    7.8s
[Parallel(n_jobs=4)]: Done 357 tasks      | elapsed:   17.7s
[Parallel(n_jobs=4)]: Done 600 out of 600 | elapsed:   29.7s finished


In [14]:
test_auc_lgbm = roc_auc_score(y_val, test_pred_lgbm)
test_auc_xgb = roc_auc_score(y_val, test_pred_xgb)
test_auc_rdf = roc_auc_score(y_val, test_pred_rdf)
print(f"Test AUC LGBM: {test_auc_lgbm}\n Test AUC XGB: {test_auc_xgb}\n Test AUC RDF: {test_auc_rdf}")

Test AUC LGBM: 0.6990493443942094
 Test AUC XGB: 0.7004403892023273
 Test AUC RDF: 0.6270050933079852


In [ ]:
test_df_ready = pd.read_parquet("/kaggle/input/data-tour-new/test_df_ready.parquet")

In [ ]:
test_preds_proba = model_xgb.predict_proba(test_df_ready[X_train.columns], iteration_range=(0, model_xgb.best_iteration))[:, 1]

In [ ]:
test_preds_proba_df = pd.DataFrame({"target": test_preds_proba})
submission_df = pd.concat([test_df_ready ["id_x_rn"], test_preds_proba_df], axis = 1)
submission_df.rename(columns={"id_x_rn": "id"}, inplace=True)
submission_df

In [ ]:
submission_df.to_parquet("sample_submission.parquet")

In [15]:
X_train_meta = pd.DataFrame({
    "pred_xgb": train_pred_xgb,
    "pred_lgbm": train_pred_lgbm,
    "pred_rdf" : train_pred_rdf
})

X_test_meta = pd.DataFrame({
    "pred_xgb": test_pred_xgb,
    "pred_lgbm": test_pred_lgbm,
    "pred_rdf" : test_pred_rdf
})


In [ ]:
new_features = ["id", "rn", "enc_paym_1", "enc_paym_2"]

In [ ]:
X_train_meta[new_features] = X_train[new_features].reset_index(drop = True)
X_test_meta[new_features] = X_val[new_features].reset_index(drop = True)

In [ ]:
X_train_meta

In [ ]:
X_train_meta.isna().sum()

In [16]:
y_train_meta = y_train.reset_index(drop = True)
y_train_meta

0           0
1           0
2           0
3           0
4           0
           ..
16776836    0
16776837    0
16776838    0
16776839    0
16776840    0
Name: flag, Length: 16776841, dtype: int8

In [23]:
y_test_meta = y_val.reset_index(drop = True)
y_test_meta

0         0
1         0
2         0
3         1
4         0
         ..
882987    0
882988    0
882989    0
882990    0
882991    0
Name: flag, Length: 882992, dtype: int8

In [17]:
start = time()
meta_model = SGDClassifier(
    loss="log_loss",   # équivalent à régression logistique
    max_iter=1000, 
    tol=1e-3,
    random_state=42
)
meta_model.fit(X_train_meta, y_train_meta)
end = time()

print(f"Duration: {end - start}")

Duration: 29.84692406654358


In [26]:
start = time()
model_lgbm = lgb.LGBMClassifier(boosting_type= 'goss', num_leaves= 128, max_depth= 7, 
                                learning_rate= 0.05, n_estimators= 3000,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_alpha=0.3,
    reg_lambda=2,
    min_child_samples=30,
    class_weight=(1 / y_test_meta.value_counts(normalize=True)).to_dict(),
    random_state=42, verbose = -1)

model_lgbm.fit(
    X_train_meta, y_train_meta,
    eval_set=[(X_test_meta, y_test_meta)],
    eval_metric='auc',
    callbacks=[
        early_stopping(stopping_rounds=150),
        log_evaluation(period=100)
    ]
)
end = time()

print(f"Duration: {end - start}")

Training until validation scores don't improve for 150 rounds
[100]	valid_0's auc: 0.70607	valid_0's binary_logloss: 0.62339
[200]	valid_0's auc: 0.706106	valid_0's binary_logloss: 0.623035
Early stopping, best iteration is:
[137]	valid_0's auc: 0.70611	valid_0's binary_logloss: 0.623257
Duration: 391.4013385772705


In [27]:
train_preds_meta = model_lgbm.predict_proba(X_train_meta)[:, 1]

In [ ]:
train_auc_meta = roc_auc_score(y_train_meta, train_preds_meta)
print(f"Train AUC meta: {train_auc_meta}\n")

In [20]:
test_preds_meta = meta_model.predict_proba(X_test_meta)[:, 1]

In [22]:
test_auc_meta = roc_auc_score(y_val_meta, test_preds_meta)
print(f"Train AUC meta: {test_auc_meta}\n")

Train AUC meta: 0.7017213315505818



In [ ]:
# Get the unique values and their counts
unique_values, counts = np.unique(train_preds_meta, return_counts=True)

# Print the results
for value, count in zip(unique_values, counts):
    print(f"{value} occurs {count} times")